## CNN: Fine-Tunning com HDC

### Imports

In [1]:
from modules import utils, globals
import torch
from modules import encoders
from binhd.classifiers import BinHD
from modules.cifake import Cifake
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.calibration import LabelEncoder
from modules.encoders import RecordEncoder
from sklearn.metrics import classification_report

### Carregando features e labels do modelo pré-treinado

In [2]:
import h5py

with h5py.File('features_labels.h5', 'r') as h5f:
    features = h5f['features'][:]
    labels = h5f['labels'][:]

np.save('features.npy', features)
np.save('labels.npy', labels)
print(f"Features salvas em '{'features.npy'}'")
print(f"Labels salvas em '{'labels.npy'}'")


Features salvas em 'features.npy'
Labels salvas em 'labels.npy'


In [3]:
features = np.load('features.npy', mmap_mode='r')
labels = np.load('labels.npy', mmap_mode='r')

### Instanciando a classe Cifake

In [4]:
cifake = Cifake()
# Verifica as amostras e classes
print(f"Número de samples: {len(cifake.samples)}")
print(f"Classes encontradas: {cifake.features}")

Número de samples: 10000
Classes encontradas:         feat_0    feat_1    feat_2    feat_3    feat_4    feat_5    feat_6  \
0     3.566299  0.552224  0.323557  0.032866  0.075987  0.785853  0.072742   
1     1.045814  0.843926  0.407755  0.868205  0.230863  0.250009  1.471078   
2     0.727950  1.614394  0.743747  0.509086  0.833209  0.972774  3.097703   
3     0.315343  0.429487  0.122323  0.047726  0.000000  0.120156  0.363570   
4     1.472784  1.147796  0.094612  2.631185  0.294509  1.003880  0.260441   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.224842  0.840820  0.170251  0.691287  2.081031  0.373396  1.113104   
9996  2.091663  0.845579  0.376688  0.166369  0.096375  0.246661  0.662203   
9997  1.052457  0.773135  0.966160  1.023654  1.183315  1.353256  0.256763   
9998  0.564769  0.919356  1.239231  0.629036  3.866544  0.299065  0.863112   
9999  0.059795  1.072452  0.775502  1.368680  1.582446  1.353043  0.481853   

        feat_7   

### Valores mínimo e máximo das features numéricas

In [5]:
min_val, max_val = cifake.get_min_max_values()
print(min_val, max_val)

0.0 10.250330924987793


### Definindo hiperparâmetros

In [6]:
dimension = 10000
num_levels = 100
low = min_val
high = max_val
oper = "bind"

### Definindo X e y

In [7]:
X = cifake.features
print(X.shape)
print(X.dtypes.unique())

y = cifake.labels
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))


(10000, 512)
[dtype('float32')]


### Carregando o modelo BinHD

In [8]:
model = BinHD(dimension, cifake.num_classes)
print(X.dtypes)

feat_0      float32
feat_1      float32
feat_2      float32
feat_3      float32
feat_4      float32
             ...   
feat_507    float32
feat_508    float32
feat_509    float32
feat_510    float32
feat_511    float32
Length: 512, dtype: object


### Record Encoder

O RecordEncoder é responsável por codificar entradas contínuas em representações hiperdimensionais binárias ou densas. Ele recebe como entrada a dimensionalidade de saída (out_features), que define o tamanho dos vetores hiperdimensionais gerados. Além disso, utiliza o parâmetro size, que representa o número de atributos de entrada, e levels, que determina quantos níveis serão usados para discretizar os dados. Os parâmetros low e high definem os limites inferior e superior dos valores de entrada.

In [9]:
record_encoder = RecordEncoder(
            out_features=dimension,
            size=X.shape[1], 
            levels=num_levels,
            low=low,
            high=high
        )


A função run_encoders processa os dados em lotes, onde os dados de entrada são convertidos em tensores, codificados pelo record_encoder, e armazenados em uma lista para serem concatenados no final, retornando todos os vetores codificados.

In [10]:
def run_encoders(X, device, record_encoder):

    assert isinstance(X, pd.DataFrame), "X precisa ser um DataFrame"
    assert not X.isnull().values.any(), "X contém valores NaN"
    assert device in ["cuda", "cpu"], f"Dispositivo inválido: {device}"

    encoded_batches = []

    record_encoder.to(device)
    record_encoder.eval()

    num_samples = len(X)

    with torch.no_grad():
        for i in range(0, num_samples, globals.BATCH_SIZE):
            end = min(i + globals.BATCH_SIZE, num_samples)

            x_batch_np = X.iloc[i:end].values.astype(np.float32)

            try:
                x_batch_tensor = torch.tensor(x_batch_np).to(device)
            except Exception as e:
                print(f"Erro ao converter batch {i}-{end} para tensor: {e}")
                continue

            try:
                encoded = record_encoder(x_batch_tensor)
                encoded_batches.append(encoded.cpu())
            except Exception as e:
                print(f"Erro ao codificar batch {i}-{end}: {e}")
                continue

            print(f"\rProcessando amostras {i} até {end}", end='', flush=True)

    if encoded_batches != []:
        encoded_all = torch.cat(encoded_batches, dim=0)
    else:
        raise RuntimeError("Nenhum batch foi codificado com sucesso.")

    return encoded_all


### Separando em treino e teste e rodando o encoder

In [11]:
X_record_encoder = run_encoders(X, globals.DEVICE, record_encoder)
labels = torch.tensor(y).to(globals.DEVICE)

y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)

X_train, X_test, y_train, y_test = train_test_split(X_record_encoder, labels, test_size=0.3, random_state = 0)

Processando amostras 9992 até 10000

/tmp/ipykernel_1024/2993563554.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)


### Treinando o modelo

In [12]:
with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test.to(torch.int8))  

y_pred_np = np.array(predictions)
y_test_np = np.array(y_test) 


/tmp/ipykernel_1024/3941384730.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_pred_np = np.array(predictions)
/tmp/ipykernel_1024/3941384730.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_test_np = np.array(y_test)


### Resultados e Conclusões

O modelo HDC (BinHD) foi usado como classificador final sobre features previamente extraídas de uma CNN pré-treinada. Para realizar uma comparação justa tentou-se utilizar 10000 imagens como entrada mas por limitações da máquina que possui 8Gbytes de memória RAM não foi possível e a entrada foi diminuída para 5000 imagens.

O modelo HDC teve uma piora considerável em relação à CNN original. Isso pode ser devido a ajustes dos hiperparâmetros, mas ao mesmo tempo não creio que justifica um resultado tão baixo, talvez existam erros na implementação do modelo.

As dificuldades para esse modelo foram as mesmas das do trabalho anterior, limitações da máquina em questão. Por ter apenas 8 Gbytes de RAM foram necessárias abordagem para que o Kernel não quebrasse e parasse a execução. Dentre essas abordagens o processamento em batch e utilizar uma biblioteca que salvasse parcialmente os arquivos na parte da extração das features. Quanto ao treinamento, mesmo que tenham sido utilizadas metade das entradas do outro modelo a execução foi mais rápida.


In [13]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.67      0.00      0.01      1487
           1       0.50      1.00      0.67      1513

    accuracy                           0.51      3000
   macro avg       0.59      0.50      0.34      3000
weighted avg       0.59      0.51      0.34      3000

